# Loading the required packages

In [5]:
%matplotlib notebook
import numpy as np
import cvxopt
import cvxopt.glpk

In [6]:
cvxopt.glpk.ilp?

In [13]:
## To help create the huge matrix

#  If the condition is true, it does nothing and your program just continues to execute. 
#But if the assert condition evaluates to false, it raises an AssertionError exception with an optional error message.


def unravel(i,j,k,size=4):
    n2 = size*size
    assert(i>=0 and i<n2)
    assert(j>=0 and i<n2)
    assert(k>=0 and i<n2)
    
    return(k+ j*n2+ i*n2*n2)


""" not used
def ravel(l,size=3):
    n2=size*size
    assert (l>=0 and l < n2*n2*n2)
    i = l // (n2*n2)
    j = (l % (n2*n2)) // n2
    k = l - i*n2*n2 - j*n2
    return ((i,j,k))
"""

' not used\ndef ravel(l,size=3):\n    n2=size*size\n    assert (l>=0 and l < n2*n2*n2)\n    i = l // (n2*n2)\n    j = (l % (n2*n2)) // n2\n    k = l - i*n2*n2 - j*n2\n    return ((i,j,k))\n'

In [65]:
## create constraint matrix

size=4
n2=size*size
A=np.zeros((4*n2*n2,n2*n2*n2))
A.shape

(1024, 4096)

In [66]:
## line constraints: only one number per line
c=0
for k in range(n2): ## for all numbers
    for j in range(n2): ## for all columns
        for i in range(n2): 
            A[c,unravel(i,j,k)] = 1 ## only one number k on line i
            A[c+n2*n2,unravel(j,i,k)] = 1 ##only one number k on line j
            A[c+2*n2*n2,unravel(j,k,i)] = 1 ## number k only once in the entire table
            f=0                             ## number k only once in a subsquare
            for m in range(size):
                for n in range(size):
                    A[c//n2+3*n2*n2+n2*f,unravel((i//size + size*m) , (j//size +size*n) ,k)] = 1
                    f+=1
        c += 1
        
       
        
print("Total number of constraints=",c)

def testA(A,c,size=4):
    n2 = size*size
    for n in range(c):
        if (np.sum(A[n,])!=n2):
            print("error on line", n)
            break
    print("All constraints OK")
    return

testA(A,c,4)

Total number of constraints= 256
All constraints OK


In [67]:
## add the fixed numbers constraints

# # important to note - the indices range from 0 to 8 and not from 1 to 9 and same is the case with alphabets
ls = [(0,0,8),(0,1,15),(0,3,12),(0,9,10),(0,15,6),(1,3,10),(1,7,15),(1,11,11),(1,12,7),(1,13,4),(1,14,13),
      (2,0,11),(2,2,4),(2,6,13),(2,7,6),(2,9,7),(2,12,0),(2,14,5),
      (3,0,1),(3,7,0),(3,8,3),(3,10,9),(3,11,2),
      (4,5,1),(4,6,15),(4,7,13),(4,9,3),(4,10,0),(4,13,14),(4,14,7),(4,15,4),
      (5,1,1),(5,3,6),(5,7,12),(5,9,11),(5,12,10),(5,14,3),
      (6,1,12),(6,3,13),(6,6,6),(6,7,3),(6,9,5),(6,12,9),(6,13,2),
      (7,0,9),(7,2,3),(7,3,4),(7,4,14),(7,6,2),(7,10,7),(7,11,13),
      (8,4,5),(8,5,7),(8,9,8),(8,11,12),(8,12,3),(8,13,0),(8,15,10),
      (9,2,14),(9,3,2),(9,6,4),(9,8,7),(9,9,1),(9,12,15),(9,14,6),
      (10,1,5),(10,3,3),(10,6,8),(10,8,9),(10,12,14),(10,14,12),
      (11,0,7),(11,1,0),(11,2,6),(11,5,12),(11,6,9),(11,8,13),(11,9,14),(11,10,3),
      (12,4,13),(12,5,14),(12,7,4),(12,8,0),(12,15,2),
      (13,1,7),(13,3,8),(13,6,12),(13,8,4),(13,9,2),(13,13,11),(13,15,5),
      (14,1,2),(14,2,9),(14,3,14),(14,4,11),(14,8,5),(14,12,4),
      (15,0,6),(15,6,7),(15,12,1),(15,14,8),(15,15,3)]
      
for lenconst in range(len(ls)):
    i,j,k = ls[lenconst]
    new = np.zeros((1,A.shape[1]))
    new[0,unravel(i,j,k)] = 1
    A = np.append(A,new,axis=0)
            
print("A.shape=", A.shape)

A.shape= (1126, 4096)


In [68]:
## solving
from cvxopt import matrix

b=matrix(np.ones(A.shape[0])) ## set partition
c=matrix(np.zeros(A.shape[1])) ## zero cost
G=matrix(np.zeros(A.shape))
h=matrix(np.zeros(A.shape[0]))
binary=np.array(range(A.shape[1]))
I=set(binary)
B=set(binary)
Aeq = matrix(A)
(status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))

In [69]:
(status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=Aeq,b=b,B=set(range(A.shape[1])))

In [70]:
# checking the status before printing
status

'optimal'

# printing the output

In [74]:
## print solution
def printsol(sol):
    sep3="+-------+-------+-------+-------+"
    for i in range(n2):
        if (i%4 == 0):
            print(sep3)
        print("|",end='')
        for j in range(n2):
            for k in range(n2):
                if (sol[unravel(i,j,k)]==1):
                    if k>9:
                        print(chr(65-10+k),end='')
                    else:
                        print(k,end='')
            if (j%4 ==3):
                print("|",end='')
            else:
                print(" ",end='')
        print("")
    print(sep3)
        
printsol(solution)
          

+-------+-------+-------+-------+
|8 F 0 C|1 B 5 7|E A D 4|2 3 9 6|
|3 6 2 A|8 9 E F|1 0 5 B|7 4 D C|
|B E 4 9|2 3 D 6|F 7 C 8|0 A 5 1|
|1 D 5 7|C 4 A 0|3 6 9 2|B 8 F E|
+-------+-------+-------+-------+
|2 B 8 5|9 1 F D|A 3 0 6|C E 7 4|
|E 1 F 6|7 8 0 C|2 B 4 9|A 5 3 D|
|0 C 7 D|4 A 6 3|8 5 1 E|9 2 B F|
|9 A 3 4|E 5 2 B|C F 7 D|8 6 1 0|
+-------+-------+-------+-------+
|4 9 D 1|5 7 B E|6 8 F C|3 0 2 A|
|C 8 E 2|3 0 4 A|7 1 B 5|F D 6 9|
|F 5 A 3|6 D 8 1|9 4 2 0|E 7 C B|
|7 0 6 B|F C 9 2|D E 3 A|5 1 4 8|
+-------+-------+-------+-------+
|5 3 B F|D E 1 4|0 9 8 7|6 C A 2|
|A 7 1 8|0 F C 9|4 2 6 3|D B E 5|
|D 2 9 E|B 6 3 8|5 C A 1|4 F 0 7|
|6 4 C 0|A 2 7 5|B D E F|1 9 8 3|
+-------+-------+-------+-------+
